In [4]:
import pandas as pd
import os
from os.path import join
# from bs4 import BeautifulSoup
# import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [5]:
data_dir = "..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp"

In [6]:
files = []
for file in os.listdir(data_dir):
    files.append(join(data_dir, file))

In [145]:
admissions = pd.read_csv(files[0], usecols=['subject_id', 'hadm_id','dischtime'])
admissions.head()

,subject_id,hadm_id,dischtime
0,10000032,22595853,2180-05-07 17:15:00
1,10000032,22841357,2180-06-27 18:49:00
2,10000032,25742920,2180-08-07 17:50:00
3,10000032,29079034,2180-07-25 17:55:00
4,10000068,25022803,2160-03-04 06:26:00


In [148]:
diagnoses = pd.read_csv(files[1])
diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


## Prescriptions

In [497]:
medication = pd.read_csv(files[-5], dtype={'ndc':str}, nrows=100)
medication.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route'],
      dtype='object')

In [505]:
import pandas as pd
import numpy as np

# assuming this is your data
medications = pd.DataFrame({
    'ndc': ['ndc1', 'ndc2', 'ndc3', 'ndc1', 'ndc2'],
    'drug': ['drug1', 'drug2', 'drug3', 'drug1', 'drug2']
})

ndc_atc_map = pd.DataFrame({
    'NDC': ['ndc1', 'ndc1', 'ndc2', 'ndc2', 'ndc3', 'ndc3'],
    'atc5': ['atc1', 'atc2', 'atc3', 'atc4', 'atc5', 'atc6']
})

def get_random_atc5(ndc):
    atc5_codes = ndc_atc_map[ndc_atc_map['NDC'] == ndc]['atc5']
    return np.random.choice(atc5_codes)

medications['atc5'] = medications['ndc'].apply(get_random_atc5)

print(medications)

    ndc   drug  atc5
0  ndc1  drug1  atc2
1  ndc2  drug2  atc4
2  ndc3  drug3  atc6
3  ndc1  drug1  atc1
4  ndc2  drug2  atc3


In [513]:
import sys
if "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\ehr_preprocess" not in sys.path:
    sys.path.append("C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\ehr_preprocess")
from ehr_preprocess.preprocessors import mimic4

In [508]:
np.random.choice([])

ValueError: 'a' cannot be empty unless no samples are taken

In [344]:
medication = pd.read_csv(files[-5], dtype={'ndc':str}, usecols=['ndc', 'drug'])
medication = medication.rename(columns={'ndc':'CONCEPT'})

In [67]:
def get_code_lengths(medication):
    medication['code_length'] = medication['CONCEPT'].map(lambda x: len(str(x)))
    return medication
def handle_11_digit_codes(medication):
    ndc_11_digit_mask = medication['code_length']==11
    medication.loc[ndc_11_digit_mask, 'CONCEPT'] = medication.loc[ndc_11_digit_mask, 'CONCEPT'].astype(str).map(
        lambda x: x[:5] + '-' + x[6:9] + '-' + x[9:])
    return medication
def handle_shorter_codes(medication):
    short_codes_mask  = medication['code_length']<11
    medication.loc[short_codes_mask, 'ndc'] = medication.loc[short_codes_mask, 'ndc'].astype(str).map()


In [345]:
medication = get_code_lengths(medication)

In [346]:
medication.code_length.value_counts()

11    13498566
1      1893328
3        24814
Name: code_length, dtype: int64

In [347]:
medication[medication.CONCEPT=="0"].drug.value_counts()

Sodium Chloride 0.9%  Flush    517674
Bag                            253066
Iso-Osmotic Dextrose           248602
0.9% Sodium Chloride           116549
Insulin                        113110
                                ...  
Dacomitinib                         1
Etravirine                          1
acyclovir 400 mg tab                1
ganciclovir 500 mg cap              1
Ledipasvir/Sofosbuvir *NF*          1
Name: drug, Length: 319, dtype: int64

In [348]:
# medication_list = medication.CONCEPT.unique().tolist()

In [388]:
medication_list = medication.CONCEPT.value_counts().index.tolist()

In [492]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import inspect

class NDC_ATC_Mapper:
    def __init__(self, medications, 
                 test=True):
        self.raw_ndc_codes = medications['CONCEPT'].unique()
        
        self.all_ndc_codes = self.get_all_ndc_codes() 
        self.possible_formattings = self.get_possible_formattings()
        self.format_map = {}
        
        self.test = test

    def __call__(self):
        self.format_codes()
        self.save_formatted_codes()
        if not os.path.exists('..\\data\\helper\\ndc_to_atc\\NDC_to_ATC_mapping.csv'):
            raise ValueError('Run https://github.com/fabkury/ndc_map on the formatted NDC codes to create the mapping file.\
                             and save it in ..\\data\\helper\\ndc_to_atc\\NDC_to_ATC_mapping.csv')

    def save_formatted_codes(self):
        data = [(k, v) for k, values in self.format_map.items() for v in values]
        formatted_codes = pd.DataFrame(data, columns=['Raw_NDC', 'NDC'])
        os.makedirs('..\\data\\interim\\mimic4', exist_ok=True)
        formatted_codes.to_csv('..\\data\\interim\\mimic4\\formatted_ndc_codes.csv', index=False)

    def get_inverse_format_map(self):
        return {v: k for k, ls in self.format_map.items() for v in ls if v is not None}
    
    def map_code(self, ndc_code):
        if len(ndc_code) < 11:
            return None
        else:
            ndc_code = self.format_ndc_code(ndc_code)
            if ndc_code is None:
                return None
            atc_code = self.retrieve_code(ndc_code)
            return atc_code
        
    def format_codes(self):
        for raw_code in self.raw_ndc_codes:
            if len(raw_code) < 11:
                self.format_map[raw_code] = [None]
            else:
                self.find_suitable_format(raw_code)

    def get_possible_formattings(self,):
        def _format1a(ndc_code):
            if ndc_code[4] == '0':
                ndc_code = ndc_code[0:4] + '-' + ndc_code[5:-2] + '-' + ndc_code[-2:]
            return ndc_code
        def _format1b(ndc_code):
            if ndc_code[4] == '0':
                ndc_code = ndc_code[0:4] + '-' + ndc_code[5:-1] + '-' + ndc_code[-1:]
            return ndc_code
        def _format2a(ndc_code):
            if ndc_code[5] == '0':
                ndc_code = ndc_code[0:5] + '-' + ndc_code[6:-2] + '-' + ndc_code[-2:]
            return ndc_code
        def _format2b(ndc_code):
            if ndc_code[5] == '0':
                ndc_code = ndc_code[0:5] + '-' + ndc_code[6:-1] + '-' + ndc_code[-1:]
            return ndc_code
        def _format3a(ndc_code):
            if ndc_code[0] == '0':
                ndc_code = ndc_code[1:5] + '-' + ndc_code[5:-2] + '-' + ndc_code[-2:]
            return ndc_code
        def _format3aa(ndc_code):
            if ndc_code[0] == '0':
                ndc_code = ndc_code[1:5] + '-' + ndc_code[5:-1] + '-' + ndc_code[-1:]
            return ndc_code
        def _format3b(ndc_code):
            if ndc_code[0] == '0':
                ndc_code = ndc_code[1:6] + '-' + ndc_code[6:-2] + '-' + ndc_code[-2:]
            return ndc_code
        def _format3ba(ndc_code):
            if ndc_code[0] == '0':
                ndc_code = ndc_code[1:6] + '-' + ndc_code[6:-1] + '-' + ndc_code[-1:]
            return ndc_code
        def _format4a(ndc_code):
            if ndc_code[-2] == '0':
                ndc_code = ndc_code[0:4] + '-' + ndc_code[4:-2] + '-' + ndc_code[-1:]
            return ndc_code
        def _format4b(ndc_code):
            if ndc_code[-2] == '0':
                ndc_code = ndc_code[0:5] + '-' + ndc_code[5:-2] + '-' + ndc_code[-1:]
            return ndc_code
        return [method for name, method in locals().items() if name.startswith('_format')]
    
    def find_suitable_format(self, raw_code):
        formatted_code = self.check_exact_match(raw_code)
        if formatted_code:
            self.format_map[raw_code] = [formatted_code]
        else:
            formatted_code = self.check_partial_match(raw_code)
            if formatted_code:
                self.format_map[raw_code] = [formatted_code]
            else:
                self.format_map[raw_code] = [self.apply_format(raw_code, formatting) for formatting in self.possible_formattings]
                self.format_map[raw_code] = [code for code in self.format_map[raw_code] if code!=raw_code]
    
    def check_exact_match(self, raw_code):
        for formatting in self.possible_formattings:
            try:
                formatted_code = self.apply_format(raw_code, formatting)
                if self.check_format_exact(formatted_code):
                    return formatted_code
            except Exception:
                continue

    def check_partial_match(self, raw_code):
        for formatting in self.possible_formattings:
            try:
                formatted_code = self.apply_format(raw_code, formatting)
                if self.check_format_partial(formatted_code):
                    return formatted_code
            except Exception:
                continue

    def apply_format(self, raw_code, formatting):
        return formatting(raw_code)
    
    def check_format_exact(self, ndc_code):
        return ndc_code in self.all_ndc_codes.values
    
    def check_format_partial(self, ndc_code):
        first_nine = self.all_ndc_codes.str.startswith(ndc_code[:9]).any()
        firs_eight = self.all_ndc_codes.str.startswith(ndc_code[:8]).any()
        first_seven = self.all_ndc_codes.str.startswith(ndc_code[:7]).any()
        return first_nine or firs_eight or first_seven
    
    @staticmethod
    def get_all_ndc_codes():
        ndc1 = pd.read_csv("..\\data\\helper\\ndc_to_atc\\ndc2atc_level4.csv").NDC
        ndc2 = pd.read_csv("..\\data\\helper\\ndc_to_atc\\NDC_codes.csv").ndcpackagecode
        return pd.concat([ndc1, ndc2]).drop_duplicates()
    


In [493]:
mapper = NDC_ATC_Mapper(medication_list[:10])

{'self': <__main__.NDC_ATC_Mapper object at 0x0000022A3A8DA850>, '_format1a': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format1a at 0x0000022A328A7430>, '_format1b': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format1b at 0x0000022A3F5904C0>, '_format2a': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format2a at 0x0000022A3F5903A0>, '_format2b': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format2b at 0x0000022A3F590AF0>, '_format3a': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format3a at 0x0000022A3F590550>, '_format3aa': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format3aa at 0x0000022A3F590CA0>, '_format3b': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format3b at 0x0000022A2E7CDF70>, '_format3ba': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format3ba at 0x0000022A2E7CDAF0>, '_format4a': <function NDC_ATC_Mapper.get_possible_formattings.<locals>._format4a a

In [469]:
[method for method in inspect.getmembers(mapper, predicate=inspect.ismethod) if method[0].startswith('_format')]

[]

In [448]:
mapper()

Run https://github.com/fabkury/ndc_map on the formatted NDC codes to create the mapping file.


In [436]:
mapper.format_codes()

KeyboardInterrupt: 

In [428]:
format_map = mapper.format_map

In [432]:
format_map

KeyError: 0

In [433]:
inv_format_map = {v: k for k, ls in format_map.items() for v in ls if v is not None}

In [434]:
inv_format_map

{'0338-0049-04': '00338004904',
 '63323-262-01': '63323026201',
 '0409-6729-24': '00409672924',
 '0904-2244-61': '00904224461',
 '0002-7510-01': '00002751001',
 '0338-0117-04': '00338011704',
 '0904-5165-61': '00904516561',
 '0088-2220-33': '00088222033',
 '0406-0552-62': '00406055262',
 '0338-3552-48': '00338355248',
 '0409-4902-34': '00409490234',
 '51079-002-20': '51079000220',
 '51079-255-20': '51079025520',
 '0409-1283-31': '00409128331',
 '0338-0049-03': '00338004903',
 '0338-0049-38': '00338004938',
 '0338-0049-02': '00338004902',
 '0338-0553-18': '00338055318',
 '0904-1988-61': '00904198861',
 '0574-7050-50': '00574705050',
 '66758-160-13': '66758016013',
 '0409-1985-30': '00409198530',
 '5539-0004-01': '55390000401',
 '5539-00040-1': '55390000401',
 '55390-004-01': '55390000401',
 '55390-0040-1': '55390000401',
 '5539-00004-1': '55390000401',
 '55390-0004-1': '55390000401',
 '0002-8215-01': '00002821501',
 '0904-4040-73': '00904404073',
 '0781-3057-14': '00781305714',
 '0338-0

In [402]:
all_ndc_codes = mapper.get_all_ndc_codes()

In [ ]:
from tqdm import tqdm
for ndc_code in tqdm(medication_list):
    if not mapper.find_suitable_format(ndc_code):
        print('No suitable format found for', ndc_code)

In [397]:
import json
import ast

In [398]:
with open("..\\data\\helper\\ndc_to_atc\\ndc2rxnorm_mapping.txt", 'r') as f:
    dictionary = ast.literal_eval(f.read())

In [401]:
len(dictionary.keys())

4204

In [377]:
def enter_code(driver, ndc_code):
    driver.get('https://www.hipaaspace.com/medical-billing/crosswalk-services/ndc-to-atc-mapping/')  # Open the page
    search_field = driver.find_element(By.NAME, 'tbxSearchRequest')  # Replace with actual id of search field
    search_field.send_keys(ndc_code)
    search_field.send_keys(Keys.RETURN)
def read_result(driver):
    # element = driver.find_element(By.CSS_SELECTOR, 'td table.table.table-bordered a.lookup_item_title')
    try:
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'lookup_item_title'))
        )
        for element in elements:
            print(element.text)
        return elements
    except TimeoutException:
        return None
    
    elements = driver.find_element(By.CLASS_NAME, 'lookup_item_title')
    for element in elements:
        print(element.text)
    text = element.text if element else None
    atc_code = text.split(' ')[0] if text else None
    
    return atc_code

def retrieve_code(driver, ndc_code):
    enter_code(driver, ndc_code)
    time.sleep(1)
    atc_code = read_result(driver)
    return atc_code
    


def map_code(driver, ndc_code):
    if len(ndc_code) < 11:
        return None
    else:
        ndc_code = format_code(ndc_code)
        atc_code = retrieve_code(driver, ndc_code)
        if atc_code:
            return atc_code
        else: # try moving the separator
            ndc_code = move_separator(ndc_code)
            atc_code = retrieve_code(driver, ndc_code)
            return atc_code
all_ndc_codes = all_ndc_codes = pd.read_csv("..\\data\\helper\\ndc_to_atc\\ndc2atc_level4.csv").ndcpackagecode.values
def check_code(ndc_code):
     return ndc_code in all_ndc_codes

AttributeError: 'DataFrame' object has no attribute 'ndcpackagecode'

In [379]:
all_ndc_codes = pd.read_csv("..\\data\\helper\\ndc_to_atc\\ndc2atc_level4.csv").NDC

In [199]:
'63323-262-01' in all_ndc_codes.values

True

In [198]:
all_ndc_codes[120000]

'63545-441-02'

In [186]:
all_ndc_codes

0         0002-0800-01
1         0002-1200-30
2         0002-1200-50
3         0002-1210-30
4         0002-1210-50
              ...     
216659    99528-606-06
216660    99528-606-12
216661    99528-606-25
216662    99528-606-45
216663    99528-606-90
Name: ndcpackagecode, Length: 216664, dtype: object

In [174]:
service = ChromeService(executable_path="C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\data\\helper\\ndc_to_atc\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service)

In [134]:
medication_list[4]

'63323026201'

In [181]:
medication_list[7]

'61958070101'

In [182]:
atc_code = map_code(driver, medication_list[7])

In [172]:
atc_code.text

AttributeError: 'NoneType' object has no attribute 'text'

In [23]:
search_field = driver.find_element(By.NAME, 'tbxSearchRequest')  # Replace with actual id of search field
search_field.send_keys(ndc_list[1])
search_field.send_keys(Keys.RETURN)
print(search_field)

<selenium.webdriver.remote.webelement.WebElement (session="06a4ca2ff600443c893741d8f738f341", element="5465A8BA6902EE0F8125AEBA53724EE4_element_6")>


In [29]:
element = driver.find_element(By.CSS_SELECTOR, 'td table.table.table-bordered a.lookup_item_title')

In [30]:
text = element.text if element else None

In [36]:
text.split(' ')[0]

'B01AB01'

In [37]:
text

'B01AB01 - heparin'

In [32]:
driver.quit()

In [283]:
prescriptions[prescriptions.ndc==51079007320]

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,ndc_len,full_ndc,full_ned
2,10000032,22595853,19796602,10000032-50,50.0,P260SK,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
17,10000032,22841357,25684927,10000032-103,103.0,P8971Q,2180-06-27 08:00:00,2180-06-27 23:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
534,10000980,24947999,23938662,10000980-502,502.0,P257ZR,2190-11-07 12:00:00,2190-11-08 20:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
546,10000980,24947999,42387321,10000980-477,477.0,P257ZR,2190-11-07 11:00:00,2190-11-07 11:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
6008,10003400,26467376,34284575,10003400-577,577.0,P266TQ,2136-12-16 08:00:00,2136-12-15 20:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
6048,10003400,27296885,52397790,10003400-613,613.0,P319LM,2137-01-01 02:00:00,2137-01-01 07:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
6050,10003400,27296885,62443934,10003400-635,635.0,P51S1E,2137-01-01 12:00:00,2137-01-03 21:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO/NG,11,5107-9007-320,5107-9007-320
8610,10004764,24817563,78728141,10004764-294,294.0,P64BPW,2168-04-16 20:00:00,2168-04-16 10:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,2.0,PO/NG,11,5107-9007-320,5107-9007-320
8779,10005012,28371912,94129501,10005012-65,65.0,P859YW,2169-10-23 10:00:00,2169-10-30 19:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,1.0,PO,11,5107-9007-320,5107-9007-320
9546,10005749,24015009,39407555,10005749-337,337.0,P89RXE,2145-09-14 08:00:00,2145-09-14 08:00:00,MAIN,Furosemide,...,NaN,40,mg,1,TAB,2.0,PO/NG,11,5107-9007-320,5107-9007-320


In [273]:
prescriptions['full_ndc'] = prescriptions.ndc.astype(str).str.zfill(10)

In [274]:
prescriptions.full_ndc.value_counts()

0000000000     1272
63323026201     222
0338004904      180
0904224461      156
0409672924      152
               ... 
50111091801       1
0536108494        1
51079010320       1
50268015215       1
0338007704        1
Name: full_ndc, Length: 1091, dtype: int64

In [276]:
prescriptions['full_ndc'] = prescriptions['full_ndc'].apply(lambda x: f"{x[:4]}-{x[4:8]}-{x[8:]}")


In [277]:
prescriptions.full_ndc

0        0904-1988-61
1        0000-0000-00
2       5107-9007-320
3        0006-0227-61
4       6332-3026-201
            ...      
9995    1152-3726-808
9996     0000-0000-00
9997    6155-3012-011
9998     0000-0000-00
9999     0338-0703-41
Name: full_ndc, Length: 10000, dtype: object

In [278]:
prescriptions[prescriptions.full_ndc=="0006-0227-61"]

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,ndc_len,full_ndc,full_ned
3,10000032,22595853,20256254,10000032-32,32.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,...,NaN,400,mg,1,TAB,2.0,PO,7,0006-0227-61,0000-6022-761
24,10000032,22841357,51588577,10000032-91,91.0,P8971Q,2180-06-26 23:00:00,2180-06-27 23:00:00,MAIN,Raltegravir,...,NaN,400,mg,1,TAB,2.0,PO,7,0006-0227-61,0000-6022-761
54,10000032,25742920,82541937,10000032-243,243.0,P76JEQ,2180-08-06 03:00:00,2180-08-07 22:00:00,MAIN,Raltegravir,...,NaN,400,mg,1,TAB,2.0,PO,7,0006-0227-61,0000-6022-761
68,10000032,29079034,49582968,10000032-182,182.0,P20F7Z,2180-07-23 17:00:00,2180-07-25 22:00:00,MAIN,Raltegravir,...,NaN,400,mg,1,TAB,2.0,PO,7,0006-0227-61,0000-6022-761


In [221]:
prescriptions.ndc.value_counts()

0              79
63323026201    32
338004904      26
904224461      21
904516561      18
               ..
51079054220     1
54839224        1
904598061       1
409739172       1
310032520       1
Name: ndc, Length: 297, dtype: int64

In [230]:
len(prescriptions[prescriptions.ndc==0].drug.value_counts(
))

12

In [232]:
prescriptions[prescriptions.ndc==0].formulary_drug_cd.value_counts(
)

NACLFLUSH    37
BAG50        10
HEPBASE       7
VANCOBASE     6
VIAL          4
SW50          3
INSULIN       3
NS100CG       2
GENTBASE1     2
FRBD100       1
FRBD50        1
ISONACL       1
METRBASE      1
LEVOBASE3     1
Name: formulary_drug_cd, dtype: int64

In [224]:
prescriptions.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route'],
      dtype='object')

In [186]:
pharmacy = pd.read_csv(files[-8], nrows=1000)

In [142]:
files

['..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\admissions.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\diagnoses_icd.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\drgcodes.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_hcpcs.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_icd_diagnoses.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_icd_procedures.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_labitems.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\emar.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\emar_detail.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\hcpcsevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\labevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\microbiologyevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\omr.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\patients.csv.gz',
 '.

In [21]:
diag = pd.read_csv(files[1])
diag.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [34]:
emar = pd.read_csv(files[8], nrows=1000)
emar.head()

,subject_id,emar_id,emar_seq,parent_field_ordinal,administration_type,pharmacy_id,barcode_type,reason_for_no_barcode,complete_dose_not_given,dose_due,...,infusion_rate_unit,route,infusion_complete,completion_interval,new_iv_bag_hung,continued_infusion_in_other_location,restart_interval,side,site,non_formulary_visual_verification
0,10000032,10000032-10,10,1.1,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,10000032-10,10,1.2,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,10000032-10,10,1.3,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,10000032-10,10,1.4,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,10000032-10,10,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
emar.columns

Index(['subject_id', 'emar_id', 'emar_seq', 'parent_field_ordinal',
       'administration_type', 'pharmacy_id', 'barcode_type',
       'reason_for_no_barcode', 'complete_dose_not_given', 'dose_due',
       'dose_due_unit', 'dose_given', 'dose_given_unit',
       'will_remainder_of_dose_be_given', 'product_amount_given',
       'product_unit', 'product_code', 'product_description',
       'product_description_other', 'prior_infusion_rate', 'infusion_rate',
       'infusion_rate_adjustment', 'infusion_rate_adjustment_amount',
       'infusion_rate_unit', 'route', 'infusion_complete',
       'completion_interval', 'new_iv_bag_hung',
       'continued_infusion_in_other_location', 'restart_interval', 'side',
       'site', 'non_formulary_visual_verification'],
      dtype='object')

In [35]:
files

['..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\admissions.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\diagnoses_icd.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\drgcodes.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_hcpcs.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_icd_diagnoses.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_icd_procedures.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\d_labitems.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\emar.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\emar_detail.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\hcpcsevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\labevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\microbiologyevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\omr.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\patients.csv.gz',
 '.

In [44]:
files[13]

'..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\patients.csv.gz'

In [65]:
omr = pd.read_csv(files[12], nrows=1000)
omr.head()

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15


In [68]:
omr.sort_values(by=['subject_id', 'result_name']).head(20)

,subject_id,chartdate,seq_num,result_name,result_value
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
22,10000032,2180-05-25,1,BMI (kg/m2),18.6
25,10000032,2180-06-01,1,BMI (kg/m2),18.7
28,10000032,2180-06-22,1,BMI (kg/m2),18.9
31,10000032,2180-06-27,1,BMI (kg/m2),19.2
34,10000032,2180-06-30,1,BMI (kg/m2),18.0
37,10000032,2180-07-09,1,BMI (kg/m2),18.7
39,10000032,2180-08-06,1,BMI (kg/m2),18.2
0,10000032,2180-04-27,1,Blood Pressure,110/65
21,10000032,2180-05-25,1,Blood Pressure,106/60


In [103]:
patient

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,dob
0,10000032,F,52,2180,2014 - 2016,2180-09-09,2128-01-01
1,10000048,F,23,2126,2008 - 2010,NaN,2103-01-01
2,10000068,F,19,2160,2008 - 2010,NaN,2141-01-01
3,10000084,M,72,2160,2017 - 2019,2161-02-13,2088-01-01
4,10000102,F,27,2136,2008 - 2010,NaN,2109-01-01
...,...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN,2101-01-01
299708,19999829,F,28,2186,2008 - 2010,NaN,2158-01-01
299709,19999840,M,58,2164,2008 - 2010,2164-09-17,2106-01-01
299710,19999914,F,49,2158,2017 - 2019,NaN,2109-01-01


In [63]:
patient['dob'] = patient['anchor_year'] - patient['anchor_age']
patient['dob'] = pd.to_datetime(patient['dob'], format='%Y')
patient.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,dob
0,10000032,F,52,2180,2014 - 2016,2180-09-09,2128-01-01
1,10000048,F,23,2126,2008 - 2010,NaN,2103-01-01
2,10000068,F,19,2160,2008 - 2010,NaN,2141-01-01
3,10000084,M,72,2160,2017 - 2019,2161-02-13,2088-01-01
4,10000102,F,27,2136,2008 - 2010,NaN,2109-01-01


In [47]:
files[14:]

['..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\pharmacy.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\poe.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\poe_detail.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\prescriptions.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\procedures_icd.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\provider.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\services.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\transfers.csv.gz']

In [117]:
files[1]

'..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\hosp\\diagnoses_icd.csv.gz'

In [207]:
diagnoses = pd.read_csv(files[1], nrows=1000)
diagnoses

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
995,10002013,23745275,5,2724,9
996,10002013,23745275,6,412,9
997,10002013,24760295,1,41071,9
998,10002013,24760295,2,5180,9


In [49]:
pharmacy = pd.read_csv(files[14], nrows=100)
pharmacy.head()

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,10000032,22595853,11700683,10000032-34,2180-05-07 01:00:00,2180-05-07 22:00:00,Acetaminophen,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,...,NaN,NaN,NaN,NaN,Ongoing,36,Hours,NaN,Omnicell,NaN
1,10000032,22595853,14779570,10000032-22,2180-05-07 00:00:00,2180-05-07 22:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,...,NaN,NaN,3.0,NaN,Ongoing,36,Hours,NaN,Floor Stock Item,NaN
2,10000032,22595853,19796602,10000032-50,2180-05-08 08:00:00,2180-05-07 22:00:00,Furosemide,Unit Dose,Discontinued via patient discharge,2180-05-07 09:32:35,...,NaN,NaN,1.0,NaN,Ongoing,36,Hours,NaN,Omnicell,NaN
3,10000032,22595853,20256254,10000032-32,2180-05-07 01:00:00,2180-05-07 22:00:00,Raltegravir,Unit Dose,Discontinued via patient discharge,2180-05-07 00:09:24,...,NaN,NaN,2.0,NaN,Ongoing,36,Hours,NaN,Omnicell,NaN
4,10000032,22595853,28781051,10000032-27,2180-05-07 00:00:00,2180-05-07 22:00:00,Heparin,Unit Dose,Discontinued via patient discharge,2180-05-07 00:00:54,...,NaN,NaN,3.0,NaN,Ongoing,36,Hours,NaN,Omnicell,NaN


In [53]:
poe = pd.read_csv(files[15], nrows=100)
poe.head()

,poe_id,poe_seq,subject_id,hadm_id,ordertime,order_type,order_subtype,transaction_type,discontinue_of_poe_id,discontinued_by_poe_id,order_provider_id,order_status
0,10000032-10,10,10000032,22595853,2180-05-06 20:50:49,Lab,NaN,New,NaN,NaN,P9705S,Inactive
1,10000032-100,100,10000032,22841357,2180-06-26 22:25:43,Respiratory,Oxygen Therapy,New,NaN,NaN,P8971Q,Inactive
2,10000032-102,102,10000032,22841357,2180-06-26 22:37:06,Medications,NaN,D/C,10000032-93,NaN,P8971Q,Inactive
3,10000032-103,103,10000032,22841357,2180-06-26 22:37:06,Medications,NaN,Change,10000032-89,NaN,P8971Q,Inactive
4,10000032-104,104,10000032,22841357,2180-06-27 07:06:36,ADT orders,Admit,Change,10000032-75,NaN,P61HH1,Inactive


In [57]:
presc = pd.read_csv(files[17], nrows=1000)
presc.head()
presc.ndc.value_counts()

0              79
63323026201    32
338004904      26
904224461      21
904516561      18
               ..
51079054220     1
54839224        1
904598061       1
409739172       1
310032520       1
Name: ndc, Length: 297, dtype: int64

In [58]:
icu_files = []
icu_data_dir = "..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu"

for file in os.listdir(icu_data_dir):
    icu_files.append(join(icu_data_dir, file))

In [59]:
icu_files

['..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\caregiver.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\chartevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\datetimeevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\d_items.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\icustays.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\ingredientevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\inputevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\outputevents.csv.gz',
 '..\\data\\raw\\mimic-iv-2.2\\mimic-iv-2.2\\icu\\procedureevents.csv.gz']

In [126]:
formatted_dir = "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\data\\formatted\\mimic4_med_diag"
patients_info = pd.read_csv(join(formatted_dir, "patients_info.csv"))

In [123]:
patients_info.head()

,PID,INSURANCE,LANGUAGE,MARITALSTATUS,RACE,GENDER,DATE_OF_DEATH,DATE_OF_BIRTH
0,10000032,Medicaid,ENGLISH,WIDOWED,WHITE,NaN,NaN,NaN
1,10000068,Other,ENGLISH,SINGLE,WHITE,F,NaN,2141-01-01
2,10000084,Medicare,ENGLISH,MARRIED,WHITE,M,2161-02-13,2088-01-01
3,10000108,Other,ENGLISH,SINGLE,WHITE,M,NaN,2138-01-01
4,10000117,Other,ENGLISH,DIVORCED,WHITE,F,NaN,2126-01-01


In [124]:
patients_info.DATE_OF_BIRTH.isna().sum()

1

In [214]:
diagnose = pd.read_csv(join(formatted_dir, "concept.diagnose.csv"))

In [216]:
diagnose['second'] = diagnose.CONCEPT.map(lambda x: x[1])

In [219]:
diagnose.second.str.isnumeric().sum()

0

In [178]:
med = pd.read_csv(join(formatted_dir, "concept.prescription_medication.csv"))

In [179]:
med

,PID,ADMISSION_ID,TIMESTAMP,CONCEPT
0,10000032,22595853,2180-05-07 00:00:00,0.000000e+00
1,10000032,22595853,2180-05-08 08:00:00,5.107901e+10
2,10000032,22595853,2180-05-07 01:00:00,6.022761e+06
3,10000032,22595853,2180-05-07 00:00:00,6.332303e+10
4,10000032,22595853,2180-05-07 01:00:00,1.730682e+08
...,...,...,...,...
9071,10005817,28661809,2135-01-15 12:00:00,4.096625e+08
9072,10005817,28661809,2135-01-10 13:00:00,1.152373e+10
9073,10005817,28661809,2135-01-13 12:00:00,0.000000e+00
9074,10005817,28661809,2135-01-13 12:00:00,6.155301e+10


In [180]:
med.CONCEPT.isna().sum()

0

In [181]:
med.CONCEPT = med.CONCEPT.astype(int)

In [182]:
med.CONCEPT

0                0
1      -2147483648
2          6022761
3      -2147483648
4        173068224
           ...    
9071     409662502
9072   -2147483648
9073             0
9074   -2147483648
9075             0
Name: CONCEPT, Length: 9076, dtype: int32

In [ ]:
def map_icd9_to_icd10(self, df):
        mapping = pd.read_csv("..\\data\\helper\\diagnosis_gems_2018\\2018_I9gem.txt", delimiter='\s+', names=['icd9', 'icd10'], usecols=[0,1])

In [212]:
mapping = pd.read_csv("..\\data\\helper\\diagnosis_gems_2018\\2018_I9gem.txt", delimiter='\s+', names=['icd9', 'icd10'], usecols=[0,1])

In [210]:
mapping = mapping.set_index('icd9')['icd10'].to_dict()

In [213]:
mapping['icd9'] = 'D' + mapping['icd9']
mapping

,icd9,icd10
0,D0010,A000
1,D0011,A001
2,D0019,A009
3,D0020,A0100
4,D0021,A011
...,...,...
24855,DV9129,O30299
24856,DV9190,O30809
24857,DV9191,O30819
24858,DV9192,O30829


In [206]:
mapping

{'0010': 'A000',
 '0011': 'A001',
 '0019': 'A009',
 '0020': 'A0100',
 '0021': 'A011',
 '0022': 'A012',
 '0023': 'A013',
 '0029': 'A014',
 '0030': 'A020',
 '0031': 'A021',
 '00320': 'A0220',
 '00321': 'A0221',
 '00322': 'A0222',
 '00323': 'A0223',
 '00324': 'A0224',
 '00329': 'A0229',
 '0038': 'A028',
 '0039': 'A029',
 '0040': 'A030',
 '0041': 'A031',
 '0042': 'A032',
 '0043': 'A033',
 '0048': 'A038',
 '0049': 'A039',
 '0050': 'A050',
 '0051': 'A051',
 '0052': 'A052',
 '0053': 'A058',
 '0054': 'A053',
 '00581': 'A055',
 '00589': 'A058',
 '0059': 'A059',
 '0060': 'A060',
 '0061': 'A061',
 '0062': 'A062',
 '0063': 'A064',
 '0064': 'A065',
 '0065': 'A066',
 '0066': 'A067',
 '0068': 'A0689',
 '0069': 'A069',
 '0070': 'A070',
 '0071': 'A071',
 '0072': 'A073',
 '0073': 'A078',
 '0074': 'A072',
 '0075': 'A074',
 '0078': 'A078',
 '0079': 'A079',
 '00800': 'A044',
 '00801': 'A040',
 '00802': 'A041',
 '00803': 'A042',
 '00804': 'A043',
 '00809': 'A044',
 '0081': 'A048',
 '0082': 'A048',
 '0083': 